# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh your knowledge of SQL Queries!

## Objectives

You will be able to:

- Use `GROUP BY` statements in SQL to apply aggregate functions like: `COUNT`, `MAX`, `MIN`, and `SUM`
- Decide and perform whichever type of join is best for retrieving desired data
- Use the `HAVING` clause to compare different aggregates
- Write subqueries to decompose complex queries

## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as Pandas. By combining them, we'll be able to write queries as Python strings, and make sure that the results are always returned as a Pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3` 
* Establish a connection to the database `data.sqlite` 
* Get the `cursor` from the connection and store it in the variable `c` 

In [7]:
import sqlite3
import pandas as pd

In [13]:
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()



In [25]:
cur.execute('SELECT * FROM CUSTOMERS').fetchall()

[(103,
  'Atelier graphique',
  'Schmitt',
  'Carine ',
  '40.32.2555',
  '54, rue Royale',
  '',
  'Nantes',
  '',
  '44000',
  'France',
  1370,
  21000),
 (112,
  'Signal Gift Stores',
  'King',
  'Jean',
  '7025551838',
  '8489 Strong St.',
  '',
  'Las Vegas',
  'NV',
  '83030',
  'USA',
  1166,
  71800),
 (114,
  'Australian Collectors, Co.',
  'Ferguson',
  'Peter',
  '03 9520 4555',
  '636 St Kilda Road',
  'Level 3',
  'Melbourne',
  'Victoria',
  '3004',
  'Australia',
  1611,
  117300),
 (119,
  'La Rochelle Gifts',
  'Labrune',
  'Janine ',
  '40.67.8555',
  '67, rue des Cinquante Otages',
  '',
  'Nantes',
  '',
  '44000',
  'France',
  1370,
  118200),
 (121,
  'Baane Mini Imports',
  'Bergulfsen',
  'Jonas ',
  '07-98 9555',
  'Erling Skakkes gate 78',
  '',
  'Stavern',
  '',
  '4110',
  'Norway',
  1504,
  81700),
 (124,
  'Mini Gifts Distributors Ltd.',
  'Nelson',
  'Susan',
  '4155551450',
  '5677 Strong St.',
  '',
  'San Rafael',
  'CA',
  '97562',
  'USA',
  1165

## Basic Queries

Now, let's review basic SQL queries. In the cell below:

Write a query that gets the first name, last name, phone number, address, and credit limit for all customers in California with a credit limit greater than 25000.00. 

In [28]:
# For the first query, the boilerplate for getting 
# the query into a DataFrame has been provided for you
thing = cur.execute(
    """
        SELECT contactFirstName, contactLastName, phone, addressLine1, creditLimit
        FROM customers
        WHERE customers.state = 'CA'
            AND creditLimit >25000.0
    """
).fetchall()
df = pd.DataFrame(thing)
df.columns = [x[0] for x in cur.description]
df

,contactFirstName,contactLastName,phone,addressLine1,creditLimit
0,Susan,Nelson,4155551450,5677 Strong St.,210500
1,Julie,Murphy,6505555787,5557 North Pendale Street,64600
2,Juri,Hashimoto,6505556809,9408 Furth Circle,84600
3,Julie,Young,6265557265,78934 Hillside Dr.,90700
4,Valarie,Thompson,7605558146,361 Furth Circle,105000
5,Julie,Brown,6505551386,7734 Strong St.,105000
6,Brian,Chandler,2155554369,6047 Douglas Av.,57700
7,Sue,Frick,4085553659,3086 Ingle Ln.,77600
8,Steve,Thompson,3105553722,3675 Furth Circle,55400
9,Sue,Taylor,4155554312,2793 Furth Circle,60300


#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that gets the average credit limit per state.

In [32]:
thing2=cur.execute(
    """
        SELECT state, AVG(creditLimit) as averageCreditLimit
        FROM customers
        GROUP BY state
    """
).fetchall()
df = pd.DataFrame(thing2)
df.columns = [x[0] for x in cur.description]
df

,state,averageCreditLimit
0,,61839.726027
1,BC,89950.000000
2,CA,83854.545455
3,CT,57350.000000
4,Co. Cork,0.000000
5,Isle of Wight,93900.000000
6,MA,70755.555556
7,NH,114200.000000
8,NJ,43000.000000
9,NSW,100550.000000


#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [35]:
thing3=cur.execute(
    '''
        SELECT customerName, customerNumber, orderNumber, status, quantityOrdered
        FROM customers
        JOIN orders
        USING (customerNumber)
        JOIN orderdetails
        USING (orderNumber)
    '''
)
df = pd.DataFrame(thing3)
df.columns = [x[0] for x in cur.description]
df.head()

,customerName,customerNumber,orderNumber,status,quantityOrdered
0,Atelier graphique,103,10123,Shipped,26
1,Atelier graphique,103,10123,Shipped,34
2,Atelier graphique,103,10123,Shipped,46
3,Atelier graphique,103,10123,Shipped,50
4,Atelier graphique,103,10298,Shipped,32


#### Expected Output

<img src='images/joins.png' width='500' height='600'> 

## HAVING and ORDER BY

Now, return the customerName, customerNumber, productName, productCode, and total number ordered for any product a customer has bought 10 or more of cumulatively. Sort the rows in descending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY -- make sure you get the order of them correct!

In [57]:
thing4=cur.execute(
    '''
        SELECT customerName,customerNumber, productName, productCode, sUM(quantityOrdered)
        FROM customers
        JOIN orders
        USING (customerNumber)
        JOIN orderdetails
        USING (orderNumber)
        JOIN products
        USING (productCode)
GROUP BY customerNumber,productCode
ORDER BY sum(quantityOrdered) DESC
    '''
)
df = pd.DataFrame(thing4)
df.columns = [x[0] for x in cur.description]
df

,customerName,customerNumber,productName,productCode,sUM(quantityOrdered)
0,Euro+ Shopping Channel,141,1992 Ferrari 360 Spider red,S18_3232,308
1,Euro+ Shopping Channel,141,1958 Chevy Corvette Limited Edition,S24_2840,245
2,Euro+ Shopping Channel,141,1970 Dodge Coronet,S24_1444,197
3,Euro+ Shopping Channel,141,1957 Chevy Pickup,S12_4473,183
4,Euro+ Shopping Channel,141,2002 Chevy Corvette,S24_3432,174
...,...,...,...,...,...
2527,La Rochelle Gifts,119,1954 Greyhound Scenicruiser,S32_2509,11
2528,Tekni Collectables Inc.,328,American Airlines: B767-300,S700_1691,11
2529,Petit Auto,314,1913 Ford Model T Speedster,S18_2949,10
2530,"Extreme Desk Decorations, Ltd",412,1961 Chevrolet Impala,S24_4620,10


#### Expected Output

<img src='images/having_order.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from offices with less than 5 employees. Print the first five rows of this DataFrame. 

In [73]:
thing5=cur.execute(
    '''
        SELECT firstName,lastName, employeeNumber, officeCode
        FROM employees
        WHERE officeCode IN (
                                SELECT officeCode
                                FROM employees
                                GROUP BY officeCode
                                HAVING count(employeeNumber) < 5
                            )
        
    '''
)
df = pd.DataFrame(thing5)
df.columns = [x[0] for x in cur.description]
df

,firstName,lastName,employeeNumber,officeCode
0,William,Patterson,1088,6
1,Julie,Firrelli,1188,2
2,Steve,Patterson,1216,2
3,Foon Yue,Tseng,1286,3
4,George,Vanauf,1323,3
5,Larry,Bott,1501,7
6,Barry,Jones,1504,7
7,Andy,Fixter,1611,6
8,Peter,Marsh,1612,6
9,Tom,King,1619,6


#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!